<a href="https://colab.research.google.com/github/Pureism/Machine_Learning_kmmi/blob/master/Prediction%20Vehicle%20Volume%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREDIKSI VOLUME KENDARAAN PADA SATU JEMBATAN

Dalam kegiatan ini, kita akan berlatih pemilihan model menggunakan validasi silang sekali lagi. 

Di sini, kita akan menggunakan kumpulan data simulasi yang mewakili variabel target yaitu volume lalu lintas dalam mobil/jam melintasi jembatan kota. Inputnya adalah berbagai fitur yang dinormalisasi terkait dengan data lalu lintas seperti waktu dan volume lalu lintas pada hari sebelumnya. 

Tujuannya adalah membangun model yang memprediksi volume lalu lintas melintasi jembatan kota dengan berbagai fitur.
Dataset berisi 10.000 record, dan untuk masing-masing record, 10 atribut/fitur disertakan. 

Tujuannya adalah untuk membangun jaringan saraf dalam yang menerima 10 fitur dan memprediksi volume lalu lintas melintasi jembatan. Karena outputnya adalah parameter numerik, ini adalah masalah regresi.

In [ ]:
# import the required packages
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np
import pandas as pd
from tensorflow import random

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the dataset
folder='/content/drive/MyDrive/Other/KMMI/Data/'
X = pd.read_csv(folder + 'traffic_volume_feats.csv')
y = pd.read_csv(folder + 'traffic_volume_target.csv')

# Print the sizes of input data and output data
print("Input data size = ", X.shape)
print("Output size = ", y.shape)
# Print the range for output
print(f"Output Range = ({y['Volume'].min()}, { y['Volume'].max()})")

Input data size =  (10000, 10)
Output size =  (10000, 1)
Output Range = (0, 584)


In [ ]:
X

,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10
0,0.510028,-0.032517,-0.414008,-0.523798,-1.949610,1.157322,-1.838422,0.528099,1.242689,-1.786399
1,-0.326082,0.136930,1.309433,-0.215550,-0.142714,0.646497,1.410953,1.122434,0.749717,-0.981489
2,1.031654,1.493003,0.444896,-0.596994,1.455600,0.618361,-1.408919,0.409685,0.850719,-0.160130
3,1.114472,-1.372319,-1.959487,0.329696,1.013829,-0.272457,-0.169959,-0.695401,-1.474272,0.038814
4,-0.395354,-1.084628,1.262634,-1.458452,-0.568413,-0.663912,1.213383,0.301551,2.336896,0.899147
...,...,...,...,...,...,...,...,...,...,...
9995,-1.513654,0.275664,0.048831,0.374313,-0.362242,-0.772904,0.252449,1.180318,-0.933421,-0.530960
9996,0.685558,-1.998606,-0.126526,-0.968824,1.323761,0.793946,0.554632,-1.003434,0.774915,-0.831321
9997,-1.175462,-1.220735,-0.071382,0.410001,0.724950,-1.217239,1.808301,-0.489635,1.950857,1.185076
9998,-1.016701,0.231030,0.994693,-0.518165,1.191007,0.993987,-0.298044,-2.420217,-0.001308,-1.057017


In [ ]:
from sklearn.model_selection import train_test_split

seed = 1
np.random.seed(seed)
random.set_seed(seed)
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(X), columns=X.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, shuffle=False)

In [ ]:
# Create the function that returns the keras model 1
def build_model_1(optimizer='adam'):
    # create model 1
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model


# Create the function that returns the keras model 2
def build_model_2(optimizer='adam'):
    # create model 2
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

# Create the function that returns the keras model 3
def build_model_3(optimizer='adam'):
    # create model 3
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

In [ ]:
# define a seed for random number generator so the result will be reproducible
seed = 1
np.random.seed(seed)
random.set_seed(seed)
# determine the number of folds for k-fold cross validation
n_folds = 5
# define the list to store cross validation scores
results_1 = []
# define the possible options for the model
models = [build_model_1, build_model_2, build_model_3]
# loop over models
for i in range(len(models)):
    # build the Scikit-Learn interface for the keras model
    regressor = KerasRegressor(build_fn=models[i], epochs=100, batch_size=50, verbose=0, shuffle=False)
    # build the pipeline of transformations so for each fold trainind set will be scaled 
    # and test set will be scaled accordingly.
    model = make_pipeline(StandardScaler(), regressor)
    # define the cross validation iterator
    kfold = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
    # perform the k-fold cross validation. 
    # store the scores in result
    result = cross_val_score(model, X_train, y_train, cv=kfold)
    # add the scores to the results list 
    results_1.append(result)

In [ ]:
# Print cross validation score for each model
rank_1 = []
for i in range(len(models)):
    rank_1.append(abs(results_1[i].mean()))
    print("Model ", i+1," test error rate = ", rank_1[i])

min = np.argwhere(rank_1 == np.min(rank_1)).reshape((1))
print(f'\nModel yang terbaik adalah menggunakan {models[min[0]]}')

Model  1  test error rate =  27.81182098388672
Model  2  test error rate =  24.99327507019043
Model  3  test error rate =  25.98710060119629
Model yang terbaik adalah menggunakan <function build_model_2 at 0x7f5a7a1ba710>


In [ ]:
results_1_rank

[27.81182098388672, 24.99327507019043, 25.98710060119629]

In [ ]:
# define a seed for random number generator so the result will be reproducible
seed = 1
np.random.seed(seed)
random.set_seed(seed)
# determine the number of folds for k-fold cross validation
n_folds = 5
# define the list to store cross validation scores
results_2 = []
# define possible options for epochs and batch_size
epochs = [80, 100]
batches = [50, 25]
# loop over all possible pairs of epochs, batch_size
for i in range(len(epochs)):
    for j in range(len(batches)):
        # build the Scikit-Learn interface for the keras model
        regressor = KerasRegressor(build_fn=build_model_2, epochs=epochs[i], batch_size=batches[j], verbose=0, shuffle=False)
        # build the pipeline of transformations so for each fold trainind set will be scaled 
        # and test set will be scaled accordingly.
        model = make_pipeline(StandardScaler(), regressor)
        # define the cross validation iterator
        kfold = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
        # perform the k-fold cross validation. 
        # store the scores in result
        result = cross_val_score(model, X_train, y_train, cv=kfold)
        # add the scores to the results list 
        results_2.append(result)

In [ ]:
# Print cross validation score for each possible pair of epochs, batch_size
c = 0
for i in range(len(epochs)):
    for j in range(len(batches)):
        print("batch_size = ", batches[j],", epochs = ", epochs[i], ", Test error rate = ", abs(results_2[c].mean()))
        c += 1

In [ ]:
# Modify the function that returns the keras model 2
def build_model_2(optimizer=optimizer):
    # create model 2
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

In [ ]:
# define a seed for random number generator so the result will be reproducible
np.random.seed(seed)
random.set_seed(seed)
# determine the number of folds for k-fold cross validation
n_folds = 5
# define the list to store cross validation scores
results_3 =[]
# define the possible options for the optimizer
optimizers = ['adam', 'sgd', 'rmsprop']

# loop over optimizers
for i in range(len(optimizers)):
    optimizer=optimizers[i]
    # build the Scikit-Learn interface for the keras model
    regressor = KerasRegressor(build_fn=build_model_2, epochs=80, batch_size=50, verbose=0, shuffle=False)
    # build the pipeline of transformations so for each fold trainind set will be scaled 
    # and test set will be scaled accordingly.
    model = make_pipeline(StandardScaler(), regressor)
    # define the cross validation iterator
    kfold = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
    # perform the k-fold cross validation. 
    # store the scores in result
    result = cross_val_score(model, X_train, y_train, cv=kfold)
    # add the scores to the results list 
    results_3.append(result)

In [ ]:
# Print cross validation score for each optimizer
# SGD terbaik
for i in range(len(optimizers)):
    print("optimizer=", optimizers[i]," test error rate = ", abs(results_3[i].mean()))

In [ ]:
# Modify Model
def build_model_2():
    # create model 2
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='sgd')
    return model

# Tugas


1.   Dapatkan model terbaik dari kasus di atas
2.   Latihlah model terbaik ini
3.   Plot loss atau accuracy training dan validasi
4.   Ujilah model terlatih ini dengan data test
5.   Cetak hasilnya

